## Fourth Gen Simulation Script

In [1]:
pwd

'C:\\Users\\DoubleDanks\\BISB\\wisser\\code\\rjwlab-scripts\\saegus_project\\devel\\jupyternbs'

In [2]:
cd C:\Users\DoubleDanks\BISB\wisser\code\recreating_synbreed

C:\Users\DoubleDanks\BISB\wisser\code\recreating_synbreed


In [3]:
import simuOpt
simuOpt.setOptions(alleleType='short', optimized=True, numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
import collections as col
from saegus import breed, operators, simulate, analyze, parse, parameters
import random
import copy
import yaml
import numpy as np
import os
np.set_printoptions(suppress=True, precision=5)


genetic_map = pd.read_csv('nam_prefounders_genetic_map.txt', index_col=None,
                         sep='\t')


with open('universal_parameters.yaml', 'r') as uparms:
    u_parameters = yaml.load(uparms)

with open('general_genetic_map_parameters.yaml', 'r') as ggmap:
    general_genetic_map_params = yaml.load(ggmap)



locus_names = u_parameters['locus_names']
pos_column = u_parameters['pos_column']
allele_names = u_parameters['allele_names']
snp_to_integer = u_parameters['snp_to_integer']
integer_to_snp = u_parameters['integer_to_snp']


alleles = general_genetic_map_params['alleles']
chr_cM_positions = general_genetic_map_params['chr_cM_positions']
cM_positions = general_genetic_map_params['cM_positions']
integral_valued_loci = general_genetic_map_params['integral_valued_loci']
relative_integral_valued_loci = general_genetic_map_params['relative_integral_valued_loci']
recombination_rates = general_genetic_map_params['recombination_rates']


"""
proto_prefix = 'C:\\Users\\DoubleDanks\\BISB\\wisser\\code\\'
run_prefix = 'rs_L10_H04\\'


tassel_input_dir_prefix = proto_prefix + run_prefix + 'tassel_input\\'
tassel_output_dir_prefix = proto_prefix + run_prefix + 'tassel_output\\'
tassel_config_prefix = proto_prefix + run_prefix + 'tassel_config_files\\'
various_simulation_info_prefix = proto_prefix + run_prefix + 'simulation_data\\'
populations_prefix = proto_prefix + run_prefix + 'populations\\'
parameter_prefix = proto_prefix + run_prefix + 'simulation_parameters\\'
ind_names_prefix = proto_prefix + run_prefix + 'ind_names\\'

hapmap_file_dir = tassel_input_dir_prefix
phenos_file_dir = tassel_input_dir_prefix
kinship_file_dir = tassel_input_dir_prefix
pop_struct_file_dir = tassel_input_dir_prefix

hapmap_file_name = 'X_simulated_hapmap.txt'
phenos_file_name = 'X_phenotype_vector.txt'
kinship_file_name = 'X_kinship_matrix.pop'
pop_struct_file_name = 'X_structure_matrix.txt'

io_parameters = dict(proto_prefix=proto_prefix,
                     run_prefix=run_prefix,
                     tassel_input_dir_prefix=tassel_output_dir_prefix,
                     tassel_output_dir_prefix=tassel_output_dir_prefix,
                     tassel_config_prefix=tassel_config_prefix,
                     populations_prefix=populations_prefix,
                     parameter_prefix=parameter_prefix,
                     ind_names_prefix=ind_names_prefix,
                     hapmap_file_dir=hapmap_file_dir,
                     phenos_file_dir=phenos_file_dir,
                     kinship_file_dir=kinship_file_dir,
                     pop_struct_file_dir=pop_struct_file_dir,
                     hapmap_file_name=hapmap_file_name,
                     phenos_file_name=phenos_file_name,
                     kinship_file_name=kinship_file_name,
                     pop_struct_file_name=pop_struct_file_name
                    )

with open('test_io_parameters.yaml', 'w') as tiop:
    yaml.dump(io_parameters, tiop)
"""

"\nproto_prefix = 'C:\\Users\\DoubleDanks\\BISB\\wisser\\code\\'\nrun_prefix = 'rs_L10_H04\\'\n\n\ntassel_input_dir_prefix = proto_prefix + run_prefix + 'tassel_input\\'\ntassel_output_dir_prefix = proto_prefix + run_prefix + 'tassel_output\\'\ntassel_config_prefix = proto_prefix + run_prefix + 'tassel_config_files\\'\nvarious_simulation_info_prefix = proto_prefix + run_prefix + 'simulation_data\\'\npopulations_prefix = proto_prefix + run_prefix + 'populations\\'\nparameter_prefix = proto_prefix + run_prefix + 'simulation_parameters\\'\nind_names_prefix = proto_prefix + run_prefix + 'ind_names\\'\n\nhapmap_file_dir = tassel_input_dir_prefix\nphenos_file_dir = tassel_input_dir_prefix\nkinship_file_dir = tassel_input_dir_prefix\npop_struct_file_dir = tassel_input_dir_prefix\n\nhapmap_file_name = 'X_simulated_hapmap.txt'\nphenos_file_name = 'X_phenotype_vector.txt'\nkinship_file_name = 'X_kinship_matrix.pop'\npop_struct_file_name = 'X_structure_matrix.txt'\n\nio_parameters = dict(proto_pr

In [4]:
nam = sim.loadPopulation(u_parameters['prefounder_file_name'])
sim.tagID(nam, reset=True)
nam.setSubPopName('maize_nam_prefounders', 0)

In [5]:
selection_statistics = {
    'aggregate': {},
    'selected': {},
    'non-selected': {}
}

ind_names_for_gwas = {i: {} for i in range(u_parameters['number_of_replicates'])}
u_parameters['meta_pop_sample_sizes'] = {i: 100 for i in range(0, u_parameters['generations_of_selection']+1, 2)}

s = simulate.Truncation(u_parameters['generations_of_selection'],
                       u_parameters['generations_of_random_mating'],
                       u_parameters['operating_population_size'],
                       u_parameters['proportion_of_individuals_saved'],
                       u_parameters['overshoot_as_proportion'],
                       u_parameters['individuals_per_breeding_subpop'],
                       u_parameters['heritability'],
                       u_parameters['meta_pop_sample_sizes'],
                       u_parameters['number_of_replicates'])

In [6]:
founders = u_parameters['founders']
replicated_nam = sim.Simulator(nam, rep=2, stealPops=False)
pop = replicated_nam.extract(0)


s.generate_f_one(pop, recombination_rates, u_parameters['founders'])
s.recombinatorial_convergence(pop, recombination_rates)
s.expand_by_selfing(pop, recombination_rates)
s.interim_random_mating(pop, recombination_rates)

Generation: 0
Generation: 1	popSize: 8
Generation: 2	popSize: 4
Generation: 3	popSize: 2
Generation: 4
Initiating interim random mating for 3 generations.
Generation: 5
Generation: 6
Generation: 7


In [7]:
multipop = sim.Simulator(pop, u_parameters['number_of_replicates'])
multi_meta = sim.Simulator(nam, u_parameters['number_of_replicates'], stealPops=False)

triplet_qtl, allele_effects = parameters.assign_identical_qtl_parameters(multipop, alleles,
                                                                         integral_valued_loci, u_parameters['number_of_qtl'],
                                                                         u_parameters['allele_effect_parameters'])

assert type(triplet_qtl[0]) == type([]), "Variables are flip-flopped in return."

for repid, pop_rep in enumerate(multipop.populations()):
    pop_rep.dvars().statistics = copy.deepcopy(selection_statistics)

s.replicate_selection(multipop, multi_meta, triplet_qtl, allele_effects,
                                recombination_rates)

Initial: Sampled 100 individuals from generation 0 Replicate: 0.
Initial: Sampled 100 individuals from generation 0 Replicate: 1.
Generation: 0
Generation: 0
Generation: 1
Generation: 1
Generation: 2
Generation: 2
Generation: 3
Generation: 3
Generation: 4
Generation: 4
Generation: 5
Generation: 5
Generation: 6
Generation: 6
Generation: 7
Generation: 7
Generation: 8
Generation: 8
Generation: 9
Generation: 9
Final: Sampled 100 individuals from generation 10
Final: Sampled 100 individuals from generation 10


In [8]:
for meta_rep in multi_meta.populations():
    assert meta_rep.numSubPop() == 7, "Correct number subpopulations before removal of the dummy population"
    meta_rep.removeSubPops(0)
    assert meta_rep.numSubPop() == 6, "Correct number after removal"

In [1]:
import importlib as imp

In [9]:
for i, meta_rep in enumerate(multi_meta.populations()):
    

    meta_rep_id = str(meta_rep.dvars().rep)
    prefix = 'R' + str(meta_rep_id) + '_'
    
    meta_rep.dvars().triplet_qtl = triplet_qtl[i]
    meta_rep.dvars().allele_effects = allele_effects[i]
    frq = analyze.Frq(meta_rep, triplet_qtl[i], alleles, allele_effects[i])
    af = frq.allele_frequencies(meta_rep, range(meta_rep.totNumLoci()))
    qtalleles = frq.rank_allele_effects(meta_rep, triplet_qtl[i], alleles, allele_effects[i])
    ties = [locus for locus in range(meta_rep.totNumLoci())
            if af['minor', 'alleles'][locus] == af['major', 'alleles'][locus]]

    for st in ties:
        af['major', 'alleles'][st] = list(meta_rep.dvars().alleleFreq[st])[0]
        af['minor', 'alleles'][st] = list(meta_rep.dvars().alleleFreq[st])[1]
    major_minor_allele_conflicts = sum(np.equal(list(af['minor', 'alleles'].values()),
                                                list(af['major', 'alleles'].values())))

    assert major_minor_allele_conflicts == 0, "There is a tie in at least one locus."

    #af_table = frq.allele_frq_table(meta_rep, meta_rep.numSubPop(), af, 
     #                                                      recombination_rates, genetic_map)
    #qtaf_table = frq.qt_allele_table(qtalleles, allele_effects[i])
    
    #af_table.to_csv(various_simulation_info_prefix + prefix + 'allele_frequency_table.txt', sep=',', index=0)
    #qtaf_table.to_csv(various_simulation_info_prefix + prefix + 'qt_allele_info.txt', sep=',', index=0)
    
    #del af_table, qtaf_table



    pca = analyze.PCA(meta_rep, range(meta_rep.totNumLoci()), frq)
    

    minor_ac = pca.calculate_count_matrix(meta_rep, af['minor', 'alleles'],
                                         prefix + 'MAC.txt')

    eigendata = pca.svd(meta_rep, minor_ac)


    individual_names = {ind.ind_id: 'rs_L10_H07_R'+ meta_rep_id +'_G' +
                        str(int(ind.generation)) +
                        '_I'+str(int(ind.ind_id))
                        for ind in meta_rep.individuals()}

    ind_names_for_gwas[meta_rep_id] = individual_names

    meta_rep.save(populations_prefix + prefix + 'metapopulation.pop')
    
    names_filename = ind_names_prefix + prefix + 'individual_names.yaml'
    with open(names_filename, 'w') as name_stream:
        yaml.dump(individual_names, name_stream)


    
    

    gwas = analyze.GWAS(meta_rep, individual_names, locus_names, pos_column)
    hmap = gwas.hapmap_formatter(integer_to_snp, tassel_input_dir_prefix + prefix + 'simulated_hapmap.txt')
    phenos = gwas.trait_formatter(tassel_input_dir_prefix + prefix + 'phenotype_vector.txt')
    kinship_matrix = gwas.calc_kinship_matrix(minor_ac, af, tassel_input_dir_prefix + prefix + 'kinship_matrix.txt')
    pop_struct_matrix = gwas.population_structure_formatter(eigendata, tassel_input_dir_prefix + prefix + 'structure_matrix.txt')
    pd.DataFrame(multipop.population(i).dvars().statistics).to_csv(tassel_input_dir_prefix + prefix + 'means_and_vars.txt', sep='\t')
    analyze.generate_tassel_gwas_configs(tassel_input_dir_prefix, 
                                         tassel_output_dir_prefix, 
                                       tassel_config_prefix, 
                                       prefix, 
                                       'sim_mlm_gwas_pipeline.xml')

## G Matrix Calculated From Saegus

#### Minor Allele Count Matrix:
    saegus calculates the minor/major alleles of every generation.
    In this the minor allele count matrix corresponds to the minor
    allele in the unselected base population, G_0. The allele 
    frequencies for the G calculation are based upon the major allele
    of the unselected base population, G_0.

#### Evaluation Using R0

In [10]:
r0_mac = np.loadtxt('R0_MAC.txt', delimiter=' ')
r0_M = np.matrix(r0_mac -1)
r0_aggregate_minor_allele_frqs = np.zeros((multi_meta.population(0).totNumLoci()))
for i in range(7386):
    maj_allele = af['minor', 'alleles'][i]
    r0_aggregate_minor_allele_frqs[i] = multi_meta.population(0).dvars().alleleFreq[i][maj_allele]
r0_P = 2*(r0_aggregate_minor_allele_frqs - 0.5)

r0_denom = np.zeros((7386))
for i, probability in enumerate(r0_aggregate_minor_allele_frqs):
    r0_denom[i] = 2*probability*(1 - probability)
r0_scaling_factor = sum(r0_denom)



In [11]:
r0_P

array([-1., -1., -1., ..., -1., -1., -1.])

In [12]:
r0_scaling_factor

677.20423750000248

In [13]:
r0_Z = np.matrix(r0_M - r0_P)
r0_G = r0_Z*r0_Z.T/sum(r0_denom)

In [15]:
np.savetxt('R0_kinship.txt', r0_G, fmt='%0.5f')

#### Evaluation Using R1

In [16]:
r1_mac = np.loadtxt('R1_MAC.txt', delimiter=' ')
r1_M = np.matrix(r1_mac - 1)
r1_aggregate_minor_allele_frqs = np.zeros((multi_meta.population(1).totNumLoci()))
for i in range(7386):
    maj_allele = af['minor', 'alleles'][i]
    r1_aggregate_minor_allele_frqs[i] = multi_meta.population(1).dvars().alleleFreq[i][maj_allele]
r1_P = 2*(r1_aggregate_minor_allele_frqs - 0.5)

r1_denom = np.zeros((7386))
for i, probability in enumerate(r1_aggregate_minor_allele_frqs):
    r1_denom[i] = 2*probability*(1 - probability)
r1_scaling_factor = sum(r0_denom)




In [17]:
r1_scaling_factor

677.20423750000248

In [18]:
r1_Z = np.matrix(r1_M - r1_P)
r1_G = r1_Z*r1_Z.T/sum(r1_denom)

In [19]:
np.savetxt('R1_kinship', r1_G, fmt='%.5f')

In [ ]:
annotated_G = pd.DataFrame(G, index=[self.individual_names[ind.ind_id]
                                     for ind in
                                     self.pop.individuals()])

# Tassel example has number of individuals in the header of the G
# matrix file
header = "{}\n".format(self.pop.popSize())

cwd = os.getcwd()
file_out_path = os.path.join(cwd, kinship_filename)

if os.path.exists(file_out_path):
    os.remove(file_out_path)
with open(kinship_filename, 'a') as f:
    f.write(header)
    annotated_G.to_csv(f, sep=' ', index=True, header=False)

In [ ]:
analyze.generate_tassel_gwas_configs(tassel_input_dir_prefix, tassel_output_dir_prefix, 
                                         tassel_config_prefix, run_prefix+prefix, 'sim_mlm_gwas_pipeline.xml')

In [ ]:
numap = hmap.drop(["rs", "alleles", "chrom", "pos", "strand", 
                   "assembly", "center", "protLSID", "assayLSID",
                  "panelLSID", "QCcode"], axis=1).T

In [ ]:
numap.columns = ["M" + str(idx+1) for idx in numap.columns]

In [ ]:
numap.columns